In [ ]:
!pip -q install tensorflow==2.3.0

In [ ]:
# Basics / Data manipulation
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import zipfile
import os

# Visualization
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import skimage.io

# ML
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

%matplotlib inline

# Data
10k+ of .tiff images
*    **80%** for training 
*    **20%** for internal testing
            *  10% Validation
            *  10% Testing

# Checking if GPU is being used

In [ ]:
import tensorflow as tf

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
with zipfile.ZipFile("../input/pc-data-dataset-gen/test.zip","r") as z:
    z.extractall(".")

# Set-up NASNetMobile

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from tensorflow.keras import optimizers
#Use this to check if the GPU is configured correctly
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


from tensorflow.keras.applications import NASNetMobile

In [ ]:
# Configuration of the NASNetMobile
#conv_base = NASNetMobile(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [ ]:
model = tf.keras.models.load_model("../input/pc-nasnetmobile-with-pregen-datasets/NASNetMobile-model.h5")

# Model 
The model will have the follow configuration:
______________
1st layer: NASNetMobile (224, 224, 3) input images
______________
2nd layer: GlobalMaxPooling2D
______________
3rd layer: Dropout with learning rate = 2e-5
______________
4th layer: Denser layer x 6 that will classify the image

model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# Avoid overfitting
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(10, activation="softmax", name="fc_out"))
conv_base.trainable = True

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=["acc"],
)

In [ ]:
model.summary()

## Data Augmentation

Before training, we preprocess a little bit the image, in order to have a better perfomance on the predictions

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#sample = plt.imread("../input/panda2/train_images/0005f7aaab2800f6170c399693a96917.png")

In [ ]:
 # Creating an object that will contain all the changes that will
 # be performed randomly to the images to help the training be more robust
 image_gen = ImageDataGenerator(
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                rescale=1/255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode="nearest"
                                )

In [ ]:
print(image_gen.flow_from_directory("./test"))

In [ ]:
batch_size = 32

In [ ]:
test_image_gen = image_gen.flow_from_directory("./test",
                                                target_size=(224, 224),
                                                batch_size=batch_size,
                                                class_mode="categorical")

In [ ]:
test_image_gen.class_indices

In [ ]:
NUMBER_OF_TESTING_IMAGES = 1051

## Testing

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
test_generator = ImageDataGenerator()
test_data_generator = test_generator.flow_from_directory(
    "./test", # Put your path here
    target_size=(224, 224),
    batch_size=32,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

In [ ]:
report_class = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report_class)   

In [ ]:
report_conf = confusion_matrix(true_classes, predicted_classes)
print(report_conf)  

In [ ]:
!rm -r test